In [ ]:
N = 100
P = 2
X = array(rnorm(N*P),c(N,P))

beta_true = array(c(1/2,3/4),c(2,1))

y = X%*%beta_true + rnorm(N,0,0.01)

L = function(beta){
    return(sum((y-X%*%beta)^2)/nrow(X))
}

beta_seq = seq(-1,1,length.out=100)
beta_df = expand.grid(b1=beta_seq,b2=beta_seq)
beta_df$L = apply(beta_df[,1:2],1,function(b)L(b))

In [ ]:
library('ggplot2')
library('viridis')

ggplot(data=beta_df,mapping=aes(x=b1,y=b2,fill=L,z=sqrt(L)))+
    geom_tile()+
    scale_fill_viridis()+
    geom_contour(color='white')+
    coord_fixed()

## test our gradient calculation:

In [ ]:
library('numDeriv')

In [ ]:
beta = c(1,2)

In [ ]:
2*t(X)%*%(X%*%beta-y)/nrow(X)

In [ ]:
grad(L,beta)

In [ ]:
gradient_descent <- function(lr = 0.05, n_steps = 100) {
  path <- data.frame(beta1 = rep(NA, n_steps + 1), beta2 = rep(NA, n_steps + 1))
  beta <- c(-1,-1)#(runif(2)-1/2)*2
  path[1, ] <- beta
  for (i in 1:n_steps) {
    dLdb <- 2 * t(X) %*% (X %*% beta - y) / nrow(X)
    beta <- beta - lr * dLdb
    path[i + 1, ] <- beta
    if (sum(dLdb^2) < 1E-5) {
      break
    }
  }
  return(path[complete.cases(path),])
}


In [ ]:
library('dplyr')

In [ ]:
gd_data <- gradient_descent()
gd_data %>% tail()

In [ ]:
library('ggplot2')
library('viridis')

ggplot(data=beta_df,mapping=aes(x=b1,y=b2,fill=L,z=sqrt(L)))+
    geom_tile()+
    scale_fill_viridis()+
    geom_contour(color='white')+
    coord_fixed() + 
    geom_point(data=gd_data,mapping=aes(x=beta1,y=beta2),inherit.aes=FALSE,color='red') + 
    geom_line(data=gd_data,mapping=aes(x=beta1,y=beta2),inherit.aes=FALSE,color='red')

In [ ]:
stoch_gradient_descent <- function(lr = 0.05, n_steps = 100) {
  path <- data.frame(beta1 = rep(NA, n_steps + 1), beta2 = rep(NA, n_steps + 1))
  beta <- c(-1,-1)#(runif(2)-1/2)*2
  path[1, ] <- beta
  for (i in 1:n_steps) {
    idx <- sample(nrow(X),5)
    X_sub = X[idx,,drop=FALSE]
    y_sub = y[idx,,drop=FALSE]
    dLdb <- 2 * t(X_sub) %*% (X_sub %*% beta - y_sub) / nrow(X_sub)
    beta <- beta - lr * dLdb
    path[i + 1, ] <- beta
    if (sum(dLdb^2) < 1E-5) {
      break
    }
  }
  return(path[complete.cases(path),])
}


In [ ]:
sgd_data <- stoch_gradient_descent()
sgd_data %>% tail()

In [ ]:
ggplot(data=beta_df,mapping=aes(x=b1,y=b2,fill=L,z=sqrt(L)))+
    geom_tile()+
    scale_fill_viridis()+
    geom_contour(color='white')+
    coord_fixed() + 
    geom_point(data=sgd_data,mapping=aes(x=beta1,y=beta2),inherit.aes=FALSE,color='red') + 
    geom_line(data=sgd_data,mapping=aes(x=beta1,y=beta2),inherit.aes=FALSE,color='red')